<a href="https://colab.research.google.com/github/muralikrishnat29/object-detection-keras-retinanet/blob/master/Keras_Retinanet_Traffic_light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fizyr/keras-retinanet.git


Cloning into 'keras-retinanet'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 5590 (delta 15), reused 18 (delta 6), pack-reused 5557
Receiving objects: 100% (5590/5590), 13.35 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (3741/3741), done.


Cloned the repo in first step. Now, stepping into the directory and installing the packages

In [2]:
%cd keras-retinanet/
!pip install .


/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170463 sha256=3669e49aa70cb0a0f0b84de7d6d4a763fd578f1ce6d3f79e4dc246bbbf4c1bca
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=2a11fe8cfcb5711b267a6b33b0137b70fc3033bb739c07e51931699860524169
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [3]:
!pip install keras --upgrade

     |████████████████████████████████| 378kB 8.1MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


Installing the req

In [4]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

Importing required libraries to connect datasets from GoogleDrive and unzip files

In [0]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

Adding config parameters

In [6]:
DATASET_DRIVEID = '1-TDo3iksS0p7fh8nvxuE6CKI3lC4f7OD'
!echo $DATASET_DRIVEID
!echo /content/keras-retinanet/keras-retinanet/`echo $DATASET_DRIVEID`.zip

1-TDo3iksS0p7fh8nvxuE6CKI3lC4f7OD
/content/keras-retinanet/keras-retinanet/1-TDo3iksS0p7fh8nvxuE6CKI3lC4f7OD.zip


Adding directory config

In [0]:
DATASET_DIR = 'dataset'
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

Download and extract dataset

In [0]:
os.makedirs(DATASET_DIR, exist_ok=True)

Extract the image dataset files into dataset directory

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
!7z x /content/drive/My\ Drive/images.zip -o/content/keras-retinanet/dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/My Drive/                                  1 file, 25912536 bytes (25 MiB)

Extracting archive: /content/drive/My Drive/images.zip
--
Path = /content/drive/My Drive/images.zip
Type = zip
Physical Size = 25912536

  0%     83% 512 - daySequence100071.jpg                                Everything is Ok

Files: 772
Size:       26594732
Compressed: 25912536


Extracting annotations

In [0]:
annotations = []
classes = set([])

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  file_name = None

  for elem in root:
    if elem.tag == 'filename':
      file_name = os.path.join(DATASET_DIR, elem.text)

    if elem.tag == 'object':
      obj_name = None
      coords = []
      for subelem in elem:
        if subelem.tag == 'name':
          obj_name = subelem.text
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            coords.append(int(float(subsubelem.text)))
            #print(int(float(subsubelem.text)))
      item = [file_name] + coords + [obj_name]
      annotations.append(item)
      classes.add(obj_name)

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))

# **Training Model**
Download pretrained model and run training.

In the next cell choose one option:

download Fizyr Resnet50 pretrained model
download your custom pretrained model, to continue previous training epochs
In the last cell optionally export trained model to Google Drive.

In [12]:

PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####
#drive.mount('/content/gdrive')
#DRIVE_MODEL = '/content/gdrive/My Drive/Colab Notebooks/objdet_tensorflow_colab/resnet50_csv_10.h5'
#shutil.copy(DRIVE_MODEL, PRETRAINED_MODEL)


print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


Training the model for given datasets

In [13]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

Using TensorFlow backend.
Creating model, this may take a second...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 36) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((36,) vs (720,)).
  weight_values[i].shape))
2020-03-24 10:43:57.298767: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-24 10:43:57.359781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-24 10:43:57.360361: I tensorflow/

In [0]:
from google.colab import files
files.download("/content/keras-retinanet/classes.csv")

Creating the folder and converting to inference. 

In [16]:
!mkdir /content/keras-retinanet/inference_models
!/content/keras-retinanet/keras_retinanet/bin/convert_model.py /content/keras-retinanet/snapshots/resnet50_csv_10.h5 /content/keras-retinanet/inference_models/testModel.h5

mkdir: cannot create directory ‘/content/keras-retinanet/inference_models’: File exists
Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2020-03-24 13:17:35.236749: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-24 13:17:35.241693: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-03-24 13:17:35.241744: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 6220dee9e2c9
2020-03-24 13:17:35.241760: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 6220dee9e2c9
2020-03-24 13:17:35.241832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 418.67.0
2020-03-24 13:17:35.241896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 41